##Basic Imports

In [ ]:
import numpy as np
import pandas as pd
import os
from os import listdir
from datetime import datetime
import datetime as dt
import dateutil.parser
import matplotlib.pyplot as plt

## File Preparation

#### Getting Networkrail Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
path = listdir('/content/drive/MyDrive/DS_Project/DS_Project_Datasets/Networkrail/Delay')
print(path)
path = ['/content/drive/MyDrive/DS_Project/DS_Project_Datasets/Networkrail/Delay/' + p for p in path]
print("\n" + str(len(path)) + " files")

['2019_20_P01_all_delays.csv', '2021_22_P11_all_delays.csv', '2021_22_P09_all_delays.csv', '2021_22_P06_all_delays.csv', '2022_23_P01_all_delays.csv', '2020_21_P13_all_delays.csv', '2018_19_P11_all_delays.csv', '2021_22_P03_all_delays.csv', '2018_19_P12_all_delays.csv', '2020_21_P06_all_delays.csv', '2018_19_P02_all_delays.csv', '2018_19_P07_all_delays.csv', '2019_20_P04_all_delays.csv', '2019_20_P02_all_delays.csv', '2019_20_P08_all_delays.csv', '2020_21_P02_all_delays.csv', '2019_20_P05_all_delays.csv', '2020_21_P04_all_delays.csv', '2019_20_P06_all_delays.csv', '2021_22_P05_all_delays.csv', '2019_20_P03_all_delays.csv', '2018_19_P05_all_delays.csv', '2021_22_P04_all_delays.csv', '2019_20_P10_all_delays.csv', '2021_22_P07_all_delays.csv', '2018_19_P09_all_delays.csv', '2018_19_P13_all_delays.csv', '2020_21_P01_all_delays.csv', '2021_22_P13_all_delays.csv', '2020_21_P11_all_delays.csv', '2018_19_P10_all_delays.csv', '2020_21_P08_all_delays.csv', '2020_21_P12_all_delays.csv', '2020_21_

In [ ]:
gpath = listdir('drive/MyDrive/Networkrail_(Delays)_2021-2022/Glossary/')
print(gpath)
gpath = ['drive/MyDrive/Networkrail_(Delays)_2021-2022/Glossary/' + p for p in gpath]

['Stanox_Codes.csv', 'Incident_Reason.csv', 'Responsible_Manager.csv', 'Reactionary_Reason.csv', 'Performance_Event.csv', 'Service_Group.csv', 'Operator.csv', 'Train_Service.csv']


In [ ]:
import warnings
warnings.filterwarnings("ignore")

## 1. Most Delayed Stations per Year

In [ ]:
Masterframemap = pd.read_csv('/content/drive/MyDrive/Stationdelay.csv')
X = Masterframemap.copy()
X = pd.DataFrame(X.reset_index())
X['YEAR'] = [a[0:4] for a in X['ORIGIN_DEPARTURE_DATE'].astype(str)]

In [ ]:
X2 = X.copy()
X2['PFPI_MINUTES'] = X2.groupby(['YEAR', 'START_STANOX'])['PFPI_MINUTES'].transform('sum')
X2 = X2.drop_duplicates(subset=['YEAR', 'START_STANOX', 'PFPI_MINUTES'])

years = X2['YEAR'].unique()
years.sort()
stan = pd.read_csv(gpath[0], encoding = "ISO-8859-1").astype({"STANOX NO.": str})
spath = pd.read_csv('/content/drive/MyDrive/DS_Project_Datasets/UK_Coordinates/stanox_coordinates.csv')
spath = spath.rename(columns={'fullName':'START_STANOX'})[['START_STANOX', 'lat', 'long']]

for year in years:
  
  year_data = X2[X2['YEAR'] == year].sort_values(by='PFPI_MINUTES', ascending = False).head(10)
  year_data['START_STANOX'] = [stan.loc[stan['STANOX NO.'] == str(float(int(row)))].head(1)['FULL NAME'].values[0] for row in year_data['START_STANOX']]
  year_data = pd.merge(year_data, spath, on="START_STANOX", how='left')
  
  nextrow = year_data[['YEAR', 'START_STANOX', 'lat', 'long', 'PFPI_MINUTES']].rename(columns={'YEAR':'Year', 'START_STANOX':'Station', 'PFPI_MINUTES':'Total Delay'})[['Year', 'Station', 'Total Delay']].head(1)
  if(year == years[0]): top5 = nextrow
  else: top5 = top5.append(nextrow)

top5

,Year,Station,Total Delay
0,2018,LONDON WATERLOO,294279.5
0,2019,LEEDS,415120.5
0,2020,ELEPHANT & CASTLE (LT),650481.0
0,2021,ELEPHANT & CASTLE (LT),464064.0
0,2022,LEEDS,109325.0


In [ ]:
incident_df = pd.read_excel('drive/MyDrive/DS_Project_Datasets/Networkrail/Glossary/' + 'Historic-Delay-Attribution-Glossary (1).xlsx', sheet_name='Incident Reason').drop(labels=[365, 366], axis="index")[['Incident Reason','Incident Category Description']]
incident_df = incident_df.rename(columns={"Incident Reason": "INCIDENT_REASON", "Incident Category Description": "MAIN_INCIDENT_REASON"})
incident_df = incident_df.drop(labels=[incident_df[incident_df['INCIDENT_REASON'].isnull()].index[0]], axis="index")

In [ ]:
checkyear = 2018

checkplace = 'LONDON WATERLOO'

#for checkplace in checkplaces:

YEARPLACE = 0
for i in range(0, len(path)):
  print(" ‣ File " + str("%02d" % (i+1,)), end = ' ')
  df = pd.read_csv(path[i], low_memory=False, parse_dates=["ORIGIN_DEPARTURE_DATE"], date_parser=lambda x: pd.to_datetime(x, errors="coerce"))
  
  df = df[['ORIGIN_DEPARTURE_DATE', 'ENGLISH_DAY_TYPE', 'INCIDENT_REASON', 'START_STANOX', 'END_STANOX', 'PFPI_MINUTES']]
  df = df.dropna()
  

  df['START_STANOX'] = df['START_STANOX'].astype(float)

  number = stan[stan['FULL NAME']==checkplace]['STANOX NO.'].reset_index()['STANOX NO.'][0]
  df = df[df['START_STANOX'] == float(number)]

  df['YEAR'] = [a[0:4] for a in df['ORIGIN_DEPARTURE_DATE'].astype(str)]
  df = df[df['YEAR'] == str(checkyear)]

  if(i == 0): YEARPLACE = df.copy()
  else: YEARPLACE = YEARPLACE.append(df.copy())

M = YEARPLACE.copy().sort_values(by='PFPI_MINUTES', ascending=False).head(1)

YEARPLACE['INCIDENT_DELAY'] = YEARPLACE.groupby(['INCIDENT_REASON'])['PFPI_MINUTES'].transform('sum')

YEARPLACE = YEARPLACE.copy().sort_values(by='INCIDENT_DELAY', ascending=False).drop_duplicates(subset=('INCIDENT_REASON')).head(1)

YEARPLACE['STATION'] = checkplace
A = YEARPLACE[['STATION', 'YEAR', 'INCIDENT_REASON', 'INCIDENT_DELAY']].copy()

A = pd.merge(left=A, right=incident_df, on="INCIDENT_REASON", how = 'left')
A = A.rename(columns={'INCIDENT_DELAY':'MAIN_INCIDENT_DELAY'})
A[['STATION', 'YEAR', 'MAIN_INCIDENT_REASON', 'MAIN_INCIDENT_DELAY']]

 ‣ File 01  ‣ File 02  ‣ File 03  ‣ File 04  ‣ File 05  ‣ File 06  ‣ File 07  ‣ File 08  ‣ File 09  ‣ File 10  ‣ File 11  ‣ File 12  ‣ File 13  ‣ File 14  ‣ File 15  ‣ File 16  ‣ File 17  ‣ File 18  ‣ File 19  ‣ File 20  ‣ File 21  ‣ File 22  ‣ File 23  ‣ File 24  ‣ File 25  ‣ File 26  ‣ File 27  ‣ File 28  ‣ File 29  ‣ File 30  ‣ File 31  ‣ File 32  ‣ File 33  ‣ File 34  ‣ File 35  ‣ File 36  ‣ File 37  ‣ File 38  ‣ File 39  ‣ File 40  ‣ File 41  ‣ File 42  ‣ File 43  ‣ File 44  ‣ File 45  ‣ File 46  ‣ File 47  ‣ File 48  ‣ File 49  ‣ File 50  ‣ File 51  ‣ File 52  ‣ File 53 

,STATION,YEAR,MAIN_INCIDENT_REASON,MAIN_INCIDENT_DELAY
0,LONDON WATERLOO,2018,External fatalities and trespass,30915.5


In [ ]:
M['STATION'] = checkplace
M = M[['YEAR', 'STATION', 'ORIGIN_DEPARTURE_DATE', 'INCIDENT_REASON', 'PFPI_MINUTES']]
M = M.rename(columns={'PFPI_MINUTES':'PEAK_DELAY'})

idf = pd.read_excel('drive/MyDrive/DS_Project_Datasets/Networkrail/Glossary/' + 'Historic-Delay-Attribution-Glossary (1).xlsx', sheet_name='Incident Reason').drop(labels=[365, 366], axis="index")[['Incident Reason','Incident Reason Description']]
idf = idf.rename(columns={"Incident Reason": "INCIDENT_REASON", "Incident Reason Description": "MAIN_INCIDENT_REASON"})
idf = idf.drop(labels=[idf[idf['INCIDENT_REASON'].isnull()].index[0]], axis="index")

M = pd.merge(left=M, right=idf, on="INCIDENT_REASON", how = 'left')
del M['INCIDENT_REASON']
M

,YEAR,STATION,ORIGIN_DEPARTURE_DATE,PEAK_DELAY,MAIN_INCIDENT_REASON
0,2018,LONDON WATERLOO,2018-06-19,147.0,Technical failures below the solebar


In [ ]:
checkyear = 2019
checkplace = 'LEEDS'

#for checkplace in checkplaces:

YEARPLACE = 0
for i in range(0, len(path)):
  print(" ‣ File " + str("%02d" % (i+1,)), end = ' ')
  df = pd.read_csv(path[i], low_memory=False, parse_dates=["ORIGIN_DEPARTURE_DATE"], date_parser=lambda x: pd.to_datetime(x, errors="coerce"))
  
  df = df[['ORIGIN_DEPARTURE_DATE', 'ENGLISH_DAY_TYPE', 'INCIDENT_REASON', 'START_STANOX', 'END_STANOX', 'PFPI_MINUTES']]
  df = df.dropna()
  
  df['START_STANOX'] = df['START_STANOX'].astype(float)

  number = stan[stan['FULL NAME']==checkplace]['STANOX NO.'].reset_index()['STANOX NO.'][0]
  df = df[df['START_STANOX'] == float(number)]

  df['YEAR'] = [a[0:4] for a in df['ORIGIN_DEPARTURE_DATE'].astype(str)]
  df = df[df['YEAR'] == str(checkyear)]

  if(i == 0): YEARPLACE = df.copy()
  else: YEARPLACE = YEARPLACE.append(df.copy())

M = YEARPLACE.copy().sort_values(by='PFPI_MINUTES', ascending=False).head(1)

YEARPLACE['INCIDENT_DELAY'] = YEARPLACE.groupby(['INCIDENT_REASON'])['PFPI_MINUTES'].transform('sum')

YEARPLACE = YEARPLACE.copy().sort_values(by='INCIDENT_DELAY', ascending=False).drop_duplicates(subset=('INCIDENT_REASON')).head(1)

YEARPLACE['STATION'] = checkplace
A = YEARPLACE[['STATION', 'YEAR', 'INCIDENT_REASON', 'INCIDENT_DELAY']].copy()

A = pd.merge(left=A, right=incident_df, on="INCIDENT_REASON", how = 'left')
A = A.rename(columns={'INCIDENT_DELAY':'MAIN_INCIDENT_DELAY'})
A[['STATION', 'YEAR', 'MAIN_INCIDENT_REASON', 'MAIN_INCIDENT_DELAY']]

 ‣ File 01  ‣ File 02  ‣ File 03  ‣ File 04  ‣ File 05  ‣ File 06  ‣ File 07  ‣ File 08  ‣ File 09  ‣ File 10  ‣ File 11  ‣ File 12  ‣ File 13  ‣ File 14  ‣ File 15  ‣ File 16  ‣ File 17  ‣ File 18  ‣ File 19  ‣ File 20  ‣ File 21  ‣ File 22  ‣ File 23  ‣ File 24  ‣ File 25  ‣ File 26  ‣ File 27  ‣ File 28  ‣ File 29  ‣ File 30  ‣ File 31  ‣ File 32  ‣ File 33  ‣ File 34  ‣ File 35  ‣ File 36  ‣ File 37  ‣ File 38  ‣ File 39  ‣ File 40  ‣ File 41  ‣ File 42  ‣ File 43  ‣ File 44  ‣ File 45  ‣ File 46  ‣ File 47  ‣ File 48  ‣ File 49  ‣ File 50  ‣ File 51  ‣ File 52  ‣ File 53 

,STATION,YEAR,MAIN_INCIDENT_REASON,MAIN_INCIDENT_DELAY
0,LEEDS,2019,Traincrew causes,55550.0


In [ ]:
M['STATION'] = checkplace
M = M[['YEAR', 'STATION', 'ORIGIN_DEPARTURE_DATE', 'INCIDENT_REASON', 'PFPI_MINUTES']]
M = M.rename(columns={'PFPI_MINUTES':'PEAK_DELAY'})

idf = pd.read_excel('drive/MyDrive/DS_Project_Datasets/Networkrail/Glossary/' + 'Historic-Delay-Attribution-Glossary (1).xlsx', sheet_name='Incident Reason').drop(labels=[365, 366], axis="index")[['Incident Reason','Incident Category Description']]
idf = idf.rename(columns={"Incident Reason": "INCIDENT_REASON", "Incident Category Description": "MAIN_INCIDENT_REASON"})
idf = idf.drop(labels=[idf[idf['INCIDENT_REASON'].isnull()].index[0]], axis="index")

M = pd.merge(left=M, right=idf, on="INCIDENT_REASON", how = 'left')
del M['INCIDENT_REASON']
M

,YEAR,STATION,ORIGIN_DEPARTURE_DATE,PEAK_DELAY,MAIN_INCIDENT_REASON
0,2019,LEEDS,2019-01-06,160.0,Technical Fleet delays


In [ ]:
checkyear = 2020
checkplace = 'ELEPHANT & CASTLE (LT)'

#for checkplace in checkplaces:

YEARPLACE = 0
for i in range(0, len(path)):
  print(" ‣ File " + str("%02d" % (i+1,)), end = ' ')
  df = pd.read_csv(path[i], low_memory=False, parse_dates=["ORIGIN_DEPARTURE_DATE"], date_parser=lambda x: pd.to_datetime(x, errors="coerce"))
  
  df = df[['ORIGIN_DEPARTURE_DATE', 'ENGLISH_DAY_TYPE', 'INCIDENT_REASON', 'START_STANOX', 'END_STANOX', 'PFPI_MINUTES']]
  df = df.dropna()
  
  df['START_STANOX'] = df['START_STANOX'].astype(float)

  number = stan[stan['FULL NAME']==checkplace]['STANOX NO.'].reset_index()['STANOX NO.'][0]
  df = df[df['START_STANOX'] == float(number)]

  df['YEAR'] = [a[0:4] for a in df['ORIGIN_DEPARTURE_DATE'].astype(str)]
  df = df[df['YEAR'] == str(checkyear)]

  if(i == 0): YEARPLACE = df.copy()
  else: YEARPLACE = YEARPLACE.append(df.copy())


M = YEARPLACE.copy().sort_values(by='PFPI_MINUTES', ascending=False).head(5).sort_values(by='ORIGIN_DEPARTURE_DATE').head(1) 

YEARPLACE['INCIDENT_DELAY'] = YEARPLACE.groupby(['INCIDENT_REASON'])['PFPI_MINUTES'].transform('sum')


YEARPLACE = YEARPLACE.copy().sort_values(by='INCIDENT_DELAY', ascending=False).drop_duplicates(subset=('INCIDENT_REASON')).head(1)

YEARPLACE['STATION'] = checkplace
A = YEARPLACE[['STATION', 'YEAR', 'INCIDENT_REASON', 'INCIDENT_DELAY']].copy()

A = pd.merge(left=A, right=incident_df, on="INCIDENT_REASON", how = 'left')
A = A.rename(columns={'INCIDENT_DELAY':'MAIN_INCIDENT_DELAY'})
A[['STATION', 'YEAR', 'MAIN_INCIDENT_REASON', 'MAIN_INCIDENT_DELAY']]

 ‣ File 01  ‣ File 02  ‣ File 03  ‣ File 04  ‣ File 05  ‣ File 06  ‣ File 07  ‣ File 08  ‣ File 09  ‣ File 10  ‣ File 11  ‣ File 12  ‣ File 13  ‣ File 14  ‣ File 15  ‣ File 16  ‣ File 17  ‣ File 18  ‣ File 19  ‣ File 20  ‣ File 21  ‣ File 22  ‣ File 23  ‣ File 24  ‣ File 25  ‣ File 26  ‣ File 27  ‣ File 28  ‣ File 29  ‣ File 30  ‣ File 31  ‣ File 32  ‣ File 33  ‣ File 34  ‣ File 35  ‣ File 36  ‣ File 37  ‣ File 38  ‣ File 39  ‣ File 40  ‣ File 41  ‣ File 42  ‣ File 43  ‣ File 44  ‣ File 45  ‣ File 46  ‣ File 47  ‣ File 48  ‣ File 49  ‣ File 50  ‣ File 51  ‣ File 52  ‣ File 53 

,STATION,YEAR,MAIN_INCIDENT_REASON,MAIN_INCIDENT_DELAY
0,ELEPHANT & CASTLE (LT),2020,Train operations,643092.0


In [ ]:
M['STATION'] = checkplace
M = M[['YEAR', 'STATION', 'ORIGIN_DEPARTURE_DATE', 'INCIDENT_REASON', 'PFPI_MINUTES']]
M = M.rename(columns={'PFPI_MINUTES':'PEAK_DELAY'})

idf = pd.read_excel('drive/MyDrive/DS_Project_Datasets/Networkrail/Glossary/' + 'Historic-Delay-Attribution-Glossary (1).xlsx', sheet_name='Incident Reason').drop(labels=[365, 366], axis="index")[['Incident Reason','Incident Category Description']]
idf = idf.rename(columns={"Incident Reason": "INCIDENT_REASON", "Incident Category Description": "MAIN_INCIDENT_REASON"})
idf = idf.drop(labels=[idf[idf['INCIDENT_REASON'].isnull()].index[0]], axis="index")

M = pd.merge(left=M, right=idf, on="INCIDENT_REASON", how = 'left')
del M['INCIDENT_REASON']
M

,YEAR,STATION,ORIGIN_DEPARTURE_DATE,PEAK_DELAY,MAIN_INCIDENT_REASON
0,2020,ELEPHANT & CASTLE (LT),2020-09-15,99.0,Train operations


In [ ]:
checkyear = 2021
checkplace = 'ELEPHANT & CASTLE (LT)'

#for checkplace in checkplaces:

YEARPLACE = 0
for i in range(0, len(path)):
  print(" ‣ File " + str("%02d" % (i+1,)), end = ' ')
  df = pd.read_csv(path[i], low_memory=False, parse_dates=["ORIGIN_DEPARTURE_DATE"], date_parser=lambda x: pd.to_datetime(x, errors="coerce"))
  
  df = df[['ORIGIN_DEPARTURE_DATE', 'ENGLISH_DAY_TYPE', 'INCIDENT_REASON', 'START_STANOX', 'END_STANOX', 'PFPI_MINUTES']]
  
  df['START_STANOX'] = df['START_STANOX'].astype(float)
  df = df.dropna()
  
  number = stan[stan['FULL NAME']==checkplace]['STANOX NO.'].reset_index()['STANOX NO.'][0]
  df = df[df['START_STANOX'] == float(number)]

  df['YEAR'] = [a[0:4] for a in df['ORIGIN_DEPARTURE_DATE'].astype(str)]
  df = df[df['YEAR'] == str(checkyear)]

  if(i == 0): YEARPLACE = df.copy()
  else: YEARPLACE = YEARPLACE.append(df.copy())

M = YEARPLACE.copy().sort_values(by='PFPI_MINUTES', ascending=False).head(1)

YEARPLACE['INCIDENT_DELAY'] = YEARPLACE.groupby(['INCIDENT_REASON'])['PFPI_MINUTES'].transform('sum')

YEARPLACE = YEARPLACE.copy().sort_values(by='INCIDENT_DELAY', ascending=False).drop_duplicates(subset=('INCIDENT_REASON')).head(1)

YEARPLACE['STATION'] = checkplace
A = YEARPLACE[['STATION', 'YEAR', 'INCIDENT_REASON', 'INCIDENT_DELAY']].copy()

A = pd.merge(left=A, right=incident_df, on="INCIDENT_REASON", how = 'left')
A = A.rename(columns={'INCIDENT_DELAY':'MAIN_INCIDENT_DELAY'})
A[['STATION', 'YEAR', 'MAIN_INCIDENT_REASON', 'MAIN_INCIDENT_DELAY']]

 ‣ File 01  ‣ File 02  ‣ File 03  ‣ File 04  ‣ File 05  ‣ File 06  ‣ File 07  ‣ File 08  ‣ File 09  ‣ File 10  ‣ File 11  ‣ File 12  ‣ File 13  ‣ File 14  ‣ File 15  ‣ File 16  ‣ File 17  ‣ File 18  ‣ File 19  ‣ File 20  ‣ File 21  ‣ File 22  ‣ File 23  ‣ File 24  ‣ File 25  ‣ File 26  ‣ File 27  ‣ File 28  ‣ File 29  ‣ File 30  ‣ File 31  ‣ File 32  ‣ File 33  ‣ File 34  ‣ File 35  ‣ File 36  ‣ File 37  ‣ File 38  ‣ File 39  ‣ File 40  ‣ File 41  ‣ File 42  ‣ File 43  ‣ File 44  ‣ File 45  ‣ File 46  ‣ File 47  ‣ File 48  ‣ File 49  ‣ File 50  ‣ File 51  ‣ File 52  ‣ File 53 

,STATION,YEAR,MAIN_INCIDENT_REASON,MAIN_INCIDENT_DELAY
0,ELEPHANT & CASTLE (LT),2021,Train operations,441440.0


In [ ]:
M['STATION'] = checkplace
M = M[['YEAR', 'STATION', 'ORIGIN_DEPARTURE_DATE', 'INCIDENT_REASON', 'PFPI_MINUTES']]
M = M.rename(columns={'PFPI_MINUTES':'PEAK_DELAY'})

idf = pd.read_excel('drive/MyDrive/DS_Project_Datasets/Networkrail/Glossary/' + 'Historic-Delay-Attribution-Glossary (1).xlsx', sheet_name='Incident Reason').drop(labels=[365, 366], axis="index")[['Incident Reason','Incident Reason Description']]
idf = idf.rename(columns={"Incident Reason": "INCIDENT_REASON", "Incident Reason Description": "MAIN_INCIDENT_REASON"})
idf = idf.drop(labels=[idf[idf['INCIDENT_REASON'].isnull()].index[0]], axis="index")

M = pd.merge(left=M, right=idf, on="INCIDENT_REASON", how = 'left')
del M['INCIDENT_REASON']
M

,YEAR,STATION,ORIGIN_DEPARTURE_DATE,PEAK_DELAY,MAIN_INCIDENT_REASON
0,2021,ELEPHANT & CASTLE (LT),2021-03-02,99.0,Delays incurred on non-Network Rail running li...


In [ ]:
checkyear = 2022
checkplace = 'LEEDS'

#for checkplace in checkplaces:

YEARPLACE = 0
for i in range(0, len(path)):
  print(" ‣ File " + str("%02d" % (i+1,)), end = ' ')
  df = pd.read_csv(path[i], low_memory=False, parse_dates=["ORIGIN_DEPARTURE_DATE"], date_parser=lambda x: pd.to_datetime(x, errors="coerce"))
  
  df = df[['ORIGIN_DEPARTURE_DATE', 'ENGLISH_DAY_TYPE', 'INCIDENT_REASON', 'START_STANOX', 'END_STANOX', 'PFPI_MINUTES']]
  df = df.dropna()
  
  df['START_STANOX'] = df['START_STANOX'].astype(float)

  number = stan[stan['FULL NAME']==checkplace]['STANOX NO.'].reset_index()['STANOX NO.'][0]
  df = df[df['START_STANOX'] == float(number)]

  df['YEAR'] = [a[0:4] for a in df['ORIGIN_DEPARTURE_DATE'].astype(str)]
  df = df[df['YEAR'] == str(checkyear)]

  if(i == 0): YEARPLACE = df.copy()
  else: YEARPLACE = YEARPLACE.append(df.copy())

M = YEARPLACE.copy().sort_values(by='PFPI_MINUTES', ascending=False).head(1)

YEARPLACE['INCIDENT_DELAY'] = YEARPLACE.groupby(['INCIDENT_REASON'])['PFPI_MINUTES'].transform('sum')

YEARPLACE = YEARPLACE.copy().sort_values(by='INCIDENT_DELAY', ascending=False).drop_duplicates(subset=('INCIDENT_REASON')).head(1)

YEARPLACE['STATION'] = checkplace
A = YEARPLACE[['STATION', 'YEAR', 'INCIDENT_REASON', 'INCIDENT_DELAY']].copy()

A = pd.merge(left=A, right=incident_df, on="INCIDENT_REASON", how = 'left')
A = A.rename(columns={'INCIDENT_DELAY':'MAIN_INCIDENT_DELAY'})
A[['STATION', 'YEAR', 'MAIN_INCIDENT_REASON', 'MAIN_INCIDENT_DELAY']]

 ‣ File 01  ‣ File 02  ‣ File 03  ‣ File 04  ‣ File 05  ‣ File 06  ‣ File 07  ‣ File 08  ‣ File 09  ‣ File 10  ‣ File 11  ‣ File 12  ‣ File 13  ‣ File 14  ‣ File 15  ‣ File 16  ‣ File 17  ‣ File 18  ‣ File 19  ‣ File 20  ‣ File 21  ‣ File 22  ‣ File 23  ‣ File 24  ‣ File 25  ‣ File 26  ‣ File 27  ‣ File 28  ‣ File 29  ‣ File 30  ‣ File 31  ‣ File 32  ‣ File 33  ‣ File 34  ‣ File 35  ‣ File 36  ‣ File 37  ‣ File 38  ‣ File 39  ‣ File 40  ‣ File 41  ‣ File 42  ‣ File 43  ‣ File 44  ‣ File 45  ‣ File 46  ‣ File 47  ‣ File 48  ‣ File 49  ‣ File 50  ‣ File 51  ‣ File 52  ‣ File 53 

,STATION,YEAR,MAIN_INCIDENT_REASON,MAIN_INCIDENT_DELAY
0,LEEDS,2022,Traincrew causes,31728.0


In [ ]:
M['STATION'] = checkplace
M = M[['YEAR', 'STATION', 'ORIGIN_DEPARTURE_DATE', 'INCIDENT_REASON', 'PFPI_MINUTES']]
M = M.rename(columns={'PFPI_MINUTES':'PEAK_DELAY'})

idf = pd.read_excel('drive/MyDrive/DS_Project_Datasets/Networkrail/Glossary/' + 'Historic-Delay-Attribution-Glossary (1).xlsx', sheet_name='Incident Reason').drop(labels=[365, 366], axis="index")[['Incident Reason','Incident Reason Description']]
idf = idf.rename(columns={"Incident Reason": "INCIDENT_REASON", "Incident Reason Description": "MAIN_INCIDENT_REASON"})
idf = idf.drop(labels=[idf[idf['INCIDENT_REASON'].isnull()].index[0]], axis="index")

M = pd.merge(left=M, right=idf, on="INCIDENT_REASON", how = 'left')
del M['INCIDENT_REASON']
M

,YEAR,STATION,ORIGIN_DEPARTURE_DATE,PEAK_DELAY,MAIN_INCIDENT_REASON
0,2022,LEEDS,2022-02-20,117.0,Severe flooding beyond that which could be mit...
